In [ ]:
# E-COMMERCE SHOPPING ASSISTANT (JUPYTER NOTEBOOK, USING EXTERNAL CSV)

import os
from datetime import datetime

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

# 👇 Your CSV path
DATA_PATH = r"C:\Users\hassa\Desktop\E-commerce-Shopping-Assistant\E-commerce Shopping Assistant\data\products.csv"
ORDERS_PATH = os.path.join(os.path.dirname(DATA_PATH), "orders.csv")


def find_column(df, candidate_names, required=False, default=None):
    """
    Try to find a column whose name matches one of candidate_names (case-insensitive).
    """
    cols_lower = {c.lower(): c for c in df.columns}
    for cand in candidate_names:
        cand = cand.lower()
        if cand in cols_lower:
            return cols_lower[cand]
    if required and default is None:
        raise ValueError(f"Required column not found. Tried: {candidate_names}")
    return default


def load_and_prepare_products(path=DATA_PATH):
    df = pd.read_csv(path)

    # ---- Detect important columns ----
    id_col = find_column(df, ["product_id", "id", "sku", "productid"], required=False)
    name_col = find_column(df, ["name", "product_name", "title", "product_title"], required=True)
    category_col = find_column(df, ["category", "product_category", "department", "product_type"], required=False)
    price_col = find_column(df, ["price", "current_price", "sale_price", "retail_price", "actual_price"], required=True)
    desc_col = find_column(df, ["description", "product_description", "details", "detail",
                                "short_description", "long_description"], required=False)

    # ---- Create unified columns ----
    if id_col is not None:
        df.rename(columns={id_col: "product_id"}, inplace=True)
    else:
        df["product_id"] = range(1, len(df) + 1)

    df.rename(columns={name_col: "name", price_col: "price"}, inplace=True)

    if category_col is not None:
        df.rename(columns={category_col: "category"}, inplace=True)
    else:
        df["category"] = "Unknown"

    if desc_col is not None:
        df.rename(columns={desc_col: "description"}, inplace=True)
    else:
        df["description"] = ""

    # ---- Clean and ensure types ----
    df["name"] = df["name"].astype(str)
    df["category"] = df["category"].astype(str)
    df["description"] = df["description"].astype(str)

    df["price"] = pd.to_numeric(df["price"], errors="coerce")
    df = df.dropna(subset=["price"])  # remove rows without price

    # ---- Text field for search/recommendation ----
    df["text"] = df["name"] + " " + df["description"]

    return df


# Load data
products_df = load_and_prepare_products(DATA_PATH)
products_df.head()

# Build TF-IDF matrix from product text
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(products_df["text"])

# Precompute cosine similarity between all products
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)


def show_products(df):
    """Display key product information."""
    if df.empty:
        print("No products to show.")
        return

    cols = ["product_id", "name", "price"]
    if "category" in df.columns:
        cols.insert(2, "category")
    display(df[cols])


def search_products(query, df=products_df, top_n=5):
    """Search products based on text query."""
    query = query.strip()
    if not query:
        print("Please type something to search.")
        return
    
    query_vec = vectorizer.transform([query])
    scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
    
    top_indices = scores.argsort()[::-1][:top_n]
    results = df.iloc[top_indices].copy()
    results["score"] = scores[top_indices]
    
    print(f"Top {top_n} search results for: '{query}'")
    show_products(results)


def filter_by_category(category, df=products_df):
    """Filter products by category (case-insensitive)."""
    category = category.strip().lower()
    if "category" not in df.columns:
        print("No 'category' column available in this dataset.")
        return
    
    mask = df["category"].str.lower() == category
    results = df[mask]
    
    if results.empty:
        print(f"No products found in category: {category}")
    else:
        print(f"Products in category: {category}")
        show_products(results)


def recommend_similar(product_id, df=products_df, sim_matrix=similarity_matrix, top_n=3):
    """Recommend similar products based on product_id."""
    if product_id not in df["product_id"].values:
        print("Invalid product_id. Please choose from the table.")
        return
    
    idx = df.index[df["product_id"] == product_id][0]
    scores = sim_matrix[idx]
    
    similar_indices = scores.argsort()[::-1]
    similar_indices = [i for i in similar_indices if i != idx][:top_n]
    
    recs = df.iloc[similar_indices]
    print(f"Products similar to '{df.loc[idx, 'name']}':")
    show_products(recs)


# Simple in-memory cart
cart = []


def add_to_cart(product_id, quantity=1, df=products_df):
    """Add a product to the cart."""
    if product_id not in df["product_id"].values:
        print("Invalid product_id.")
        return
    
    # Check if item exists in cart
    for item in cart:
        if item["product_id"] == product_id:
            item["quantity"] += quantity
            print(f"Updated quantity for product_id {product_id}.")
            return
    
    product = df[df["product_id"] == product_id].iloc[0]
    cart.append({
        "product_id": product_id,
        "name": product["name"],
        "price": product["price"],
        "quantity": quantity
    })
    print(f"Added to cart: {product['name']} (x{quantity})")


def view_cart():
    """View items in the cart."""
    if not cart:
        print("Your cart is empty.")
        return
    
    df_cart = pd.DataFrame(cart)
    df_cart["total"] = df_cart["price"] * df_cart["quantity"]
    print("🛒 Your cart:")
    display(df_cart)
    print(f"Grand total: {df_cart['total'].sum()}")


def clear_cart():
    """Clear all items in the cart."""
    cart.clear()
    print("Cart cleared.")


def checkout(save_path=ORDERS_PATH):
    """Save cart to orders.csv and clear it."""
    if not cart:
        print("Your cart is empty. Add items before checkout.")
        return
    
    df_cart = pd.DataFrame(cart)
    df_cart["total"] = df_cart["price"] * df_cart["quantity"]
    df_cart["order_id"] = datetime.now().strftime("%Y%m%d%H%M%S")
    df_cart["order_timestamp"] = datetime.now().isoformat(timespec="seconds")

    cols = ["order_id", "order_timestamp", "product_id", "name", "price", "quantity", "total"]
    df_cart = df_cart[cols]

    if os.path.exists(save_path):
        df_cart.to_csv(save_path, mode="a", header=False, index=False)
    else:
        df_cart.to_csv(save_path, index=False)

    print(f"✅ Order saved to {save_path}")
    print(f"Order ID: {df_cart['order_id'].iloc[0]}")
    clear_cart()


def shopping_assistant():
    print("👋 Welcome to the E-commerce Shopping Assistant!")
    print("Using CSV:", DATA_PATH)
    
    while True:
        print("\nWhat would you like to do?")
        print("1. View sample of products")
        print("2. Search products")
        print("3. Filter by category")
        print("4. Recommend similar products")
        print("5. Add to cart")
        print("6. View cart")
        print("7. Clear cart")
        print("8. Checkout (save order)")
        print("9. Exit")
        
        choice = input("Enter choice (1-9): ").strip()
        
        if choice == "1":
            show_products(products_df.head(20))
        
        elif choice == "2":
            query = input("Search for (e.g. 'shoes', 't-shirt', 'bag'): ")
            search_products(query)
        
        elif choice == "3":
            cat = input("Enter category (exact name, e.g. 'Shoes', 'Clothing', 'Accessories'): ")
            filter_by_category(cat)
        
        elif choice == "4":
            try:
                pid = int(input("Enter product_id to get similar items: "))
                recommend_similar(pid)
            except ValueError:
                print("Please enter a valid number.")
        
        elif choice == "5":
            try:
                pid = int(input("Enter product_id to add to cart: "))
                qty = int(input("Enter quantity: "))
                add_to_cart(pid, qty)
            except ValueError:
                print("Please enter valid numbers for product_id and quantity.")
        
        elif choice == "6":
            view_cart()
        
        elif choice == "7":
            clear_cart()
        
        elif choice == "8":
            checkout()
        
        elif choice == "9":
            print("Thank you for shopping! 👋")
            break
        
        else:
            print("Invalid choice, please select 1–9.")


# Run this in a separate cell if you want it to start immediately:
# shopping_assistant()


In [2]:
shopping_assistant()

👋 Welcome to the E-commerce Shopping Assistant!
Using CSV: C:\Users\hassa\Desktop\E-commerce Shopping Assistant\data\products.csv

What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


Enter choice (1-9):  8


Your cart is empty. Add items before checkout.

What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


Enter choice (1-9):  5
Enter product_id to add to cart:  name


Please enter valid numbers for product_id and quantity.

What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


...

Top 5 search results for: '126704571.0'


,product_id,name,category,price
0,126704571.0,New Look trench coat in camel,New Look trench coat in camel,49.99
3,126704571.0,New Look trench coat in camel,New Look trench coat in camel,49.99
2,126704571.0,New Look trench coat in camel,New Look trench coat in camel,49.99
1,126704571.0,New Look trench coat in camel,New Look trench coat in camel,49.99
30837,121144566.0,ASOS LUXE Wedding embellished high neck mini d...,ASOS LUXE Wedding embellished high neck mini d...,165.00



What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


Enter choice (1-9):  9


Thank you for shopping! 👋


In [3]:
# ==============================
# SALES / ORDERS ANALYSIS SECTION
# ==============================

import matplotlib.pyplot as plt

def load_orders(path=ORDERS_PATH):
    """
    Load orders.csv if it exists, else print a friendly message.
    """
    if not os.path.exists(path):
        print(f"No orders file found at {path}")
        print("👉 Run shopping_assistant(), add items to cart, and use 'Checkout' first.")
        return None
    
    df = pd.read_csv(path)
    
    # If empty file (no rows), handle gracefully
    if df.empty:
        print("orders.csv exists but has no rows yet.")
        return None
    
    # Parse timestamp if column present
    if "order_timestamp" in df.columns:
        df["order_timestamp"] = pd.to_datetime(df["order_timestamp"], errors="coerce")
        df["order_date"] = df["order_timestamp"].dt.date
    else:
        print("Warning: 'order_timestamp' column not found in orders.csv")
    
    return df


def show_basic_kpis(orders_df):
    """
    Show basic metrics: total revenue, AOV, etc.
    """
    if orders_df is None or orders_df.empty:
        print("No order data available for KPIs.")
        return
    
    total_revenue = orders_df["total"].sum()
    total_order_lines = len(orders_df)
    unique_orders = orders_df["order_id"].nunique()
    
    order_revenue = orders_df.groupby("order_id")["total"].sum()
    aov = order_revenue.mean()
    
    unique_products_sold = orders_df["product_id"].nunique()
    
    print("===== KEY METRICS (KPIs) =====")
    print(f"💰 Total Revenue:        {total_revenue}")
    print(f"🧾 Total Order Lines:    {total_order_lines}")
    print(f"📦 Unique Orders:        {unique_orders}")
    print(f"💳 Avg Order Value (AOV): {round(aov, 2)}")
    print(f"🛍️ Unique Products Sold: {unique_products_sold}")


def plot_daily_revenue(orders_df):
    """
    Plot simple daily revenue chart.
    """
    if orders_df is None or orders_df.empty:
        print("No order data available for daily revenue chart.")
        return
    
    if "order_date" not in orders_df.columns:
        print("No 'order_date' column in orders data (check timestamp parsing).")
        return
    
    daily_revenue = (
        orders_df.groupby("order_date")["total"]
        .sum()
        .reset_index()
        .sort_values("order_date")
    )
    
    if daily_revenue.empty:
        print("No revenue data to plot.")
        return
    
    print("Daily revenue (first few rows):")
    display(daily_revenue.head())
    
    plt.figure(figsize=(10, 4))
    plt.plot(daily_revenue["order_date"], daily_revenue["total"])
    plt.title("Daily Revenue")
    plt.xlabel("Date")
    plt.ylabel("Revenue")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


def top_products_by_revenue(orders_df, top_n=10):
    """
    Show and plot top N products by revenue.
    """
    if orders_df is None or orders_df.empty:
        print("No order data available for product analysis.")
        return
    
    product_revenue = (
        orders_df
        .groupby(["product_id", "name"], as_index=False)
        .agg(
            total_revenue=("total", "sum"),
            total_quantity=("quantity", "sum")
        )
        .sort_values("total_revenue", ascending=False)
    )
    
    print(f"Top {top_n} products by revenue:")
    display(product_revenue.head(top_n))
    
    top_products = product_revenue.head(top_n)
    
    plt.figure(figsize=(10, 4))
    plt.bar(top_products["name"], top_products["total_revenue"])
    plt.title(f"Top {top_n} Products by Revenue")
    plt.xlabel("Product")
    plt.ylabel("Revenue")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()


def revenue_by_category(orders_df, products_df):
    """
    Join orders with products to compute revenue by category (if category exists).
    """
    if orders_df is None or orders_df.empty:
        print("No order data available for category analysis.")
        return
    
    # Ensure product_id types match
    orders_df = orders_df.copy()
    products_df = products_df.copy()
    
    orders_df["product_id"] = orders_df["product_id"].astype(str)
    products_df["product_id"] = products_df["product_id"].astype(str)
    
    # Try to detect category column
    cat_col = find_column(products_df, ["category", "product_category", "department", "product_type"])
    
    if cat_col is None:
        print("No category column found in products.csv, skipping category analysis.")
        return
    
    products_df.rename(columns={cat_col: "category"}, inplace=True)
    
    merged = orders_df.merge(
        products_df[["product_id", "category"]],
        on="product_id",
        how="left"
    )
    
    cat_rev = (
        merged
        .groupby("category", as_index=False)["total"]
        .sum()
        .sort_values("total", ascending=False)
    )
    
    print("Revenue by Category:")
    display(cat_rev)
    
    plt.figure(figsize=(10, 4))
    plt.bar(cat_rev["category"], cat_rev["total"])
    plt.title("Revenue by Category")
    plt.xlabel("Category")
    plt.ylabel("Revenue")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()


def run_sales_analysis():
    """
    High-level function to run the full analysis pipeline.
    """
    print("📥 Loading orders from:", ORDERS_PATH)
    orders_df = load_orders(ORDERS_PATH)
    
    if orders_df is None:
        return  # message already printed inside load_orders
    
    show_basic_kpis(orders_df)
    plot_daily_revenue(orders_df)
    top_products_by_revenue(orders_df, top_n=10)
    revenue_by_category(orders_df, products_df)


In [4]:
shopping_assistant()

👋 Welcome to the E-commerce Shopping Assistant!
Using CSV: C:\Users\hassa\Desktop\E-commerce Shopping Assistant\data\products.csv

What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


Enter choice (1-9):  5
Enter product_id to add to cart:  8
Enter quantity:  9


Invalid product_id.

What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


Enter choice (1-9):  9


Thank you for shopping! 👋


In [5]:
run_sales_analysis()

📥 Loading orders from: C:\Users\hassa\Desktop\E-commerce Shopping Assistant\data\orders.csv
No orders file found at C:\Users\hassa\Desktop\E-commerce Shopping Assistant\data\orders.csv
👉 Run shopping_assistant(), add items to cart, and use 'Checkout' first.


In [6]:
shopping_assistant()

👋 Welcome to the E-commerce Shopping Assistant!
Using CSV: C:\Users\hassa\Desktop\E-commerce Shopping Assistant\data\products.csv

What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


Enter choice (1-9):  1


,product_id,name,category,price
0,126704571.0,New Look trench coat in camel,New Look trench coat in camel,49.99
1,126704571.0,New Look trench coat in camel,New Look trench coat in camel,49.99
2,126704571.0,New Look trench coat in camel,New Look trench coat in camel,49.99
3,126704571.0,New Look trench coat in camel,New Look trench coat in camel,49.99
4,123650194.0,Stradivarius double breasted wool coat in grey,Stradivarius double breasted wool coat in grey,59.99
5,123650194.0,Stradivarius double breasted wool coat in grey,Stradivarius double breasted wool coat in grey,59.99
6,123650194.0,Stradivarius double breasted wool coat in grey,Stradivarius double breasted wool coat in grey,59.99
7,123650194.0,Stradivarius double breasted wool coat in grey,Stradivarius double breasted wool coat in grey,59.99
8,125806824.0,JDY oversized trench coat in stone,JDY oversized trench coat in stone,45.00
9,125806824.0,JDY oversized trench coat in stone,JDY oversized trench coat in stone,45.00



What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


Enter choice (1-9):  5
Enter product_id to add to cart:  2
Enter quantity:  1


...

Your cart is empty.

What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


Enter choice (1-9):  8


Your cart is empty. Add items before checkout.

What would you like to do?
1. View sample of products
2. Search products
3. Filter by category
4. Recommend similar products
5. Add to cart
6. View cart
7. Clear cart
8. Checkout (save order)
9. Exit


Enter choice (1-9):  9


Thank you for shopping! 👋


In [7]:
run_sales_analysis()

📥 Loading orders from: C:\Users\hassa\Desktop\E-commerce Shopping Assistant\data\orders.csv
No orders file found at C:\Users\hassa\Desktop\E-commerce Shopping Assistant\data\orders.csv
👉 Run shopping_assistant(), add items to cart, and use 'Checkout' first.
